In [47]:
import pandas as pd
df=pd.read_csv('train/train.csv')
df.head()

,영업일자,영업장명_메뉴명,매출수량
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102676 entries, 0 to 102675
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   영업일자      102676 non-null  object
 1   영업장명_메뉴명  102676 non-null  object
 2   매출수량      102676 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.4+ MB


In [49]:
df['영업장명_메뉴명'].nunique()

193

In [50]:
# 날짜 형식으로 변환
df['영업일자'] = pd.to_datetime(df['영업일자'])

# 영업장명, 메뉴명 분리
df[['영업장명', '메뉴명']] = df['영업장명_메뉴명'].str.split('_', n=1, expand=True)

# 필요없는 원본 컬럼 제거
df.drop(columns=['영업장명_메뉴명'], inplace=True)
df.head()

,영업일자,매출수량,영업장명,메뉴명
0,2023-01-01,0,느티나무 셀프BBQ,1인 수저세트
1,2023-01-02,0,느티나무 셀프BBQ,1인 수저세트
2,2023-01-03,0,느티나무 셀프BBQ,1인 수저세트
3,2023-01-04,0,느티나무 셀프BBQ,1인 수저세트
4,2023-01-05,0,느티나무 셀프BBQ,1인 수저세트


In [51]:
df.groupby('영업장명')['매출수량'].sum().sort_values(ascending=False).head(10)

영업장명
포레스트릿         305421
카페테리아         240850
화담숲주막         146311
담하            124917
미라시아           98071
느티나무 셀프BBQ     69786
화담숲카페          62634
연회장            28373
라그로타           17453
Name: 매출수량, dtype: int64

In [52]:
# 영업장별, 메뉴별 매출수량 합계 계산
grouped = df.groupby(['영업장명', '메뉴명'])['매출수량'].sum().reset_index()

# 매출수량 기준 내림차순 정렬
grouped_sales = grouped.sort_values(by='매출수량', ascending=False)
grouped_sales.reset_index(drop=True, inplace=True)
print(grouped_sales.head(20))

     영업장명                메뉴명   매출수량
0   포레스트릿               꼬치어묵  68733
1   화담숲주막               해물파전  60126
2   포레스트릿                떡볶이  56424
3   포레스트릿                 생수  41445
4   포레스트릿             치즈 핫도그  30402
5   카페테리아          수제 등심 돈까스  27336
6   카페테리아       단체식 18000(신)  26571
7   화담숲카페          아메리카노 ICE  22525
8   포레스트릿               코카콜라  22035
9   포레스트릿         아메리카노(ICE)  21000
10  카페테리아       단체식 13000(신)  20784
11   미라시아         브런치(대인) 주말  20426
12   미라시아         브런치(대인) 주중  19823
13  화담숲주막               병천순대  17339
14     담하                공깃밥  16448
15     담하  (단체) 황태해장국 3/27까지  15892
16  카페테리아        약 고추장 돌솥비빔밥  15606
17  포레스트릿           복숭아 아이스티  15370
18  화담숲주막           느린마을 막걸리  15187
19     담하          담하 한우 불고기  14957


In [ ]:
# 파생 컬럼 생성
df['년'] = df['영업일자'].dt.year
df['월'] = df['영업일자'].dt.month
df['일'] = df['영업일자'].dt.date
df['요일'] = df['영업일자'].dt.day_name()
df.head()

,영업일자,매출수량,영업장명,메뉴명,년,월,일,요일
0,2023-01-01,0,느티나무 셀프BBQ,1인 수저세트,2023,1,2023-01-01,Sunday
1,2023-01-02,0,느티나무 셀프BBQ,1인 수저세트,2023,1,2023-01-02,Monday
2,2023-01-03,0,느티나무 셀프BBQ,1인 수저세트,2023,1,2023-01-03,Tuesday
3,2023-01-04,0,느티나무 셀프BBQ,1인 수저세트,2023,1,2023-01-04,Wednesday
4,2023-01-05,0,느티나무 셀프BBQ,1인 수저세트,2023,1,2023-01-05,Thursday


In [ ]:
#월별 메뉴별 판매량 시각화
import seaborn as sns
import matplotlib.pyplot as plt

def plot_monthly_sales(df):
    monthly = df.groupby(['영업장명', '메뉴명', '년', '월'])['매출수량'].sum().reset_index()
    g = sns.catplot(data=monthly, x='월', y='매출수량', hue='메뉴명', col='영업장명',
                    kind='bar', col_wrap=3, height=4, aspect=1.5)
    g.set_titles("{col_name}")
    g.set_axis_labels("월", "총 매출수량")
    plt.suptitle("월별 메뉴별 판매량 (영업장별)", y=1.03)
    plt.tight_layout()
    plt.show()
